In [1]:
import pandas as pd
!pip install transformers

     |████████████████████████████████| 4.2 MB 18.3 MB/s 
     |████████████████████████████████| 84 kB 2.9 MB/s 
     |████████████████████████████████| 596 kB 45.8 MB/s 
     |████████████████████████████████| 6.6 MB 29.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
link = 'https://drive.google.com/file/d/1a1mZHqe3o1psMlsaXrqIUOe9IiYQ2gRT/view?usp=sharing' 

In [4]:
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Survey_Results_Labeled_Dec.csv') 
 
df = pd.read_csv('Survey_Results_Labeled_Dec.csv')
df

,Unnamed: 0,text,1,2,3,Label
0,Q1,"""Tucker Carlson seem to have forgotten that Ru...",0.1,0.0,0.9,negative
1,Q2,"""Analysis: Would Putin get away with invading ...",0.0,0.4,0.6,negative
2,Q3,"""Kiev, Ukraine - Ukrainian defense officials s...",0.0,0.8,0.2,neutral
3,Q4,Russia-Ukraine tensions: why Russia is putting...,0.0,0.5,0.5,neutral
4,Q5,@ManMadeMoon I don’t think it’s that deep. I t...,0.1,0.0,0.9,negative
...,...,...,...,...,...,...
97,Q96,"""@peterpomeranzev Putin doesn't want to invade...",0.0,0.2,0.8,negative
98,Q97,@JonScottFNC Ukrainians remember The Holodomor...,0.0,0.2,0.8,negative
99,Q98,‘Fake news’ may complicate Putin-Biden talks –...,0.0,0.9,0.1,neutral
100,Q99,"""Russia kept EU busy by instigating a hybrid w...",0.0,0.5,0.5,neutral


In [5]:
df['Sentiment'] = 0

In [6]:
df

,Unnamed: 0,text,1,2,3,Label,Sentiment
0,Q1,"""Tucker Carlson seem to have forgotten that Ru...",0.1,0.0,0.9,negative,0
1,Q2,"""Analysis: Would Putin get away with invading ...",0.0,0.4,0.6,negative,0
2,Q3,"""Kiev, Ukraine - Ukrainian defense officials s...",0.0,0.8,0.2,neutral,0
3,Q4,Russia-Ukraine tensions: why Russia is putting...,0.0,0.5,0.5,neutral,0
4,Q5,@ManMadeMoon I don’t think it’s that deep. I t...,0.1,0.0,0.9,negative,0
...,...,...,...,...,...,...,...
97,Q96,"""@peterpomeranzev Putin doesn't want to invade...",0.0,0.2,0.8,negative,0
98,Q97,@JonScottFNC Ukrainians remember The Holodomor...,0.0,0.2,0.8,negative,0
99,Q98,‘Fake news’ may complicate Putin-Biden talks –...,0.0,0.9,0.1,neutral,0
100,Q99,"""Russia kept EU busy by instigating a hybrid w...",0.0,0.5,0.5,neutral,0


In [7]:
from google.colab import drive
drive.mount('drive')
import sys
sys.path.append('drive/My Drive/Colab Notebooks')

Mounted at drive


In [8]:
#code from NLPiation

import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load the BERT Tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')

# The dataset class
class TheDataset(torch.utils.data.Dataset):

    def __init__(self, reviews, sentiments, tokenizer):
        self.reviews    = reviews
        self.sentiments = sentiments
        self.tokenizer  = tokenizer
        self.max_len    = tokenizer.model_max_length
  
    def __len__(self):
        return len(self.reviews)
  
    def __getitem__(self, index):
        review = str(self.reviews[index])
        sentiments = self.sentiments[index]

        encoded_review = self.tokenizer.encode_plus(
            review,
            add_special_tokens    = True,
            max_length            = 128,
            return_token_type_ids = False,
            return_attention_mask = True,
            return_tensors        = 'pt',
            padding               = "max_length",
            truncation            = True
        )

        return {
            'input_ids': encoded_review['input_ids'][0],
            'attention_mask': encoded_review['attention_mask'][0],
            'labels': torch.tensor(sentiments, dtype=torch.long)
        }

# Prepare the Train/Validation sets
#train_set_dataset = TheDataset(
    #reviews    = train_set.text.tolist(),
    #sentiments = train_set.Sentiment.tolist(),
    #tokenizer  = tokenizer,
#)

#valid_set_dataset = TheDataset(
    #reviews    = valid_set.text.tolist(),
    #sentiments = valid_set.Sentiment.tolist(),
    #tokenizer  = tokenizer,
#)

# Load the BERT model
#model = BertForSequenceClassification.from_pretrained("drive/My Drive/Colab Notebooks/epoch 4", problem_type="single_label_classification", num_labels=3)

# Freeze the whole BERT model and train just the classifier
#for name, param in model.bert.named_parameters():
    #param.requires_grad = False

# Freeze BERT except the pooler layer
#for name, param in model.bert.named_parameters():
    #if not name.startswith('pooler'):
        #param.requires_grad = False

# Freeze the first 18 layers of the BERT
#for name, param in model.bert.named_parameters():
    #if ( not name.startswith('pooler') ) and "layer.23" and "layer.22" and "layer.21" and "layer.20" and "layer.19" and "layer.18" not in name :
        #param.requires_grad = False

# The function to get the accuracy
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Define the training parameters
training_args = TrainingArguments(
    output_dir                  = 'drive/My Drive/Colab Notebooks',
    num_train_epochs            = 2,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size  = 64,
    warmup_steps                = 10000,
    learning_rate               = 5e-5,
    weight_decay                = 0.01,
    adam_beta1                  = 0.9,
    adam_beta2                  = 0.999,
    save_strategy               = "epoch",
    evaluation_strategy         = "epoch",
    eval_steps                  = 1,
    #save_total_limit            = 5,
    #metric_for_best_model       = 'eval_loss',
    load_best_model_at_end      =True
)

# Define the Huggingface Trainer object
#trainer = Trainer(
    #model           = model,
    #args            = training_args,
    #train_dataset   = train_set_dataset,
    #eval_dataset    = valid_set_dataset,
    #compute_metrics = compute_metrics,
    #callbacks       = [EarlyStoppingCallback(early_stopping_patience=3)]
#)


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [9]:
# Load the checkpoint
model = BertForSequenceClassification.from_pretrained("drive/My Drive/Colab Notebooks/epoch 2", num_labels=3)

# Make the test set ready
test_set_dataset = TheDataset(
    reviews    = df.text.tolist(),
    sentiments = df.Sentiment.tolist(),
    tokenizer  = tokenizer,
)

training_args = TrainingArguments(
    output_dir = 'drive/My Drive/Colab Notebooks',
    do_predict = True
)

trainer = Trainer(
    model           = model,
    args            = training_args,
    compute_metrics =compute_metrics,
)

PredictionOutput = trainer.predict(test_set_dataset)

***** Running Prediction *****
  Num examples = 102
  Batch size = 8


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
import numpy as np
np.argmax(PredictionOutput[0][98])

0

In [11]:
PredictionOutput[0]

array([[ 3.9677444 , -1.6481946 , -1.8766682 ],
       [-2.9736743 ,  4.5024123 , -2.5321507 ],
       [-1.9589286 , -2.5659332 ,  5.1971583 ],
       [ 5.3472123 , -2.411278  , -2.4928217 ],
       [ 4.761672  , -2.1461363 , -2.3029175 ],
       [-2.085013  ,  3.7938437 , -2.018036  ],
       [ 5.130168  , -2.2256484 , -3.0419936 ],
       [-2.2022321 , -2.5002096 ,  5.466708  ],
       [ 5.305526  , -2.696856  , -2.4243312 ],
       [-0.97443783,  2.7577417 , -1.8253481 ],
       [-2.880897  ,  3.9495988 , -1.7839353 ],
       [ 5.552987  , -2.6387792 , -2.661018  ],
       [ 4.3183436 , -1.9460115 , -1.864502  ],
       [ 5.1063504 , -2.5725205 , -2.1344085 ],
       [ 4.865454  , -2.0182147 , -2.73583   ],
       [-1.9561851 , -2.423502  ,  5.135487  ],
       [-1.6970978 , -1.9867926 ,  4.166263  ],
       [ 2.6735716 , -1.048059  , -1.9426608 ],
       [-1.4471397 , -1.6532073 ,  3.3189664 ],
       [-1.4818377 ,  2.636275  , -1.0831773 ],
       [ 5.387339  , -2.7066445 , -2.428

In [12]:
df['Bert_Large label'] = [np.argmax(i) for i in (PredictionOutput[0])]

In [13]:
df['Bert_Large label'].replace({0: "negative", 1: "neutral", 2: 'positive'}, inplace=True)

In [14]:
results = df['Bert_Large label'].value_counts()

In [15]:
results

negative    55
positive    27
neutral     20
Name: Bert_Large label, dtype: int64

In [16]:
df

,Unnamed: 0,text,1,2,3,Label,Sentiment,Bert_Large label
0,Q1,"""Tucker Carlson seem to have forgotten that Ru...",0.1,0.0,0.9,negative,0,negative
1,Q2,"""Analysis: Would Putin get away with invading ...",0.0,0.4,0.6,negative,0,neutral
2,Q3,"""Kiev, Ukraine - Ukrainian defense officials s...",0.0,0.8,0.2,neutral,0,positive
3,Q4,Russia-Ukraine tensions: why Russia is putting...,0.0,0.5,0.5,neutral,0,negative
4,Q5,@ManMadeMoon I don’t think it’s that deep. I t...,0.1,0.0,0.9,negative,0,negative
...,...,...,...,...,...,...,...,...
97,Q96,"""@peterpomeranzev Putin doesn't want to invade...",0.0,0.2,0.8,negative,0,negative
98,Q97,@JonScottFNC Ukrainians remember The Holodomor...,0.0,0.2,0.8,negative,0,negative
99,Q98,‘Fake news’ may complicate Putin-Biden talks –...,0.0,0.9,0.1,neutral,0,neutral
100,Q99,"""Russia kept EU busy by instigating a hybrid w...",0.0,0.5,0.5,neutral,0,negative


In [17]:
#delete control tweets from set
df = df.drop(33)
df = df.drop(67)

In [18]:
#change column names to make clear what is what
df = df.rename(columns={"Label":"Manual Label"})

In [19]:
#drop unnecessary column used as fake true label.
df = df.drop('Sentiment',axis=1)

In [20]:
 import sklearn
 from sklearn.metrics import balanced_accuracy_score, precision_recall_fscore_support
 b_a_Bert_Large = balanced_accuracy_score(df['Manual Label'], df['Bert_Large label'])
 p_r_f1_class_Bert_Large = precision_recall_fscore_support(df['Manual Label'], df['Bert_Large label'], average=None, labels=['negative', 'neutral', 'positive'])
 p_r_f1_Bert_Large = precision_recall_fscore_support(df['Manual Label'], df['Bert_Large label'], average='macro')

In [21]:
print(b_a_Bert_Large)
print(p_r_f1_Bert_Large)
print(p_r_f1_class_Bert_Large)

0.38689458689458683
(0.402326685660019, 0.38689458689458683, 0.3324224680112774, None)
(array([0.51851852, 0.65      , 0.03846154]), array([0.53846154, 0.28888889, 0.33333333]), array([0.52830189, 0.4       , 0.06896552]), array([52, 45,  3]))


In [ ]:
import pandas as pd
y_actu = df['Manual Label']
y_pred = df['Bert_Large label']
df_confusion = pd.crosstab(y_actu, y_pred)

In [ ]:
df_confusion

Bert_Large label,negative,neutral,positive
Manual Label,,,
negative,28,7,17
neutral,24,13,8
positive,2,0,1


In [ ]:
#Get accuracies per class order (negative, neutral, positive)
from sklearn.metrics import confusion_matrix
y_true = df['Manual Label']
y_pred = df['Bert_Large label']
matrix = confusion_matrix(y_true, y_pred)
matrix.diagonal()/matrix.sum(axis=1)

array([0.53846154, 0.28888889, 0.33333333])

In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
df.to_csv('BERT_Large_Test_Dec.csv')
!cp BERT_Large_Test_Dec.csv "/content/drive/My Drive/Colab Notebooks"

